In [ ]:

url = ""
with open("../nba_graphql/app.env") as file:
    db_source = file.read()
    url = db_source.split("DB_SOURCE=\"")[1][:-1]

In [ ]:
from pymongo import MongoClient
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
import time
client = MongoClient(url)
db = client.nba

In [ ]:
from nba_api.stats.static import teams

def lookupTeam(abbreviation):
    original = abbreviation
    if abbreviation == "SEA":
        abbreviation = "OKC"
    if abbreviation == "NOH" or abbreviation == "NOK":
        abbreviation = "NOP"
    if abbreviation == "NJN":
        abbreviation = "BKN"
    ret = [team for team in teams.get_teams() if team['abbreviation'] == abbreviation]
    if len(ret) == 0:
        print(original)
    return ret[0]


In [ ]:
# # get teams

# for team in teams.get_teams():
#     team_data = {
#         "teamID": team['id'],
#         'name': team['nickname'],
#         'abbreviation': team['abbreviation'],
#         "city": team["city"],
#     }
#     if team["abbreviation"] == "OKC":
#         team_data["alternate_abbreviations"] = ["SEA"]
#     if team["abbreviation"] == "NOP":
#         team_data["alternate_abbreviations"] = ["NOK", "NOH"]
#     if team["abbreviation"] == "BKN":
#         team_data["alternate_abbreviations"] = ["NJN"]
#     db.teams.update_many({"teamID": team['id']}, {"$set": team_data}, True)

In [ ]:
# # add players to db
# #TODO: query all players from db at once instead of one at a time
# from nba_api.stats.endpoints import commonallplayers
# from nba_api.stats.endpoints import commonplayerinfo
# from nba_api.stats.endpoints import playercareerstats
# from nba_api.stats.library.parameters import PerModeSimple

# all_players = commonallplayers.CommonAllPlayers()
# data = all_players.get_normalized_dict()
# count = 0
# for player in data['CommonAllPlayers']:
#     if player["ROSTERSTATUS"] == 1:
#         names = player["DISPLAY_FIRST_LAST"].split(" ", 1)
#         player_data = {
#             "playerID":player["PERSON_ID"],
#             "first_name":names[0],
#             "last_name":names[1],
#             "teamABR": player["TEAM_ABBREVIATION"],
#             "seasons":[],
#         }
#         player_career = playercareerstats.PlayerCareerStats(player["PERSON_ID"], per_mode36=PerModeSimple.per_game)
#         regular_seasons = player_career.get_normalized_dict()["SeasonTotalsRegularSeason"]
#         time.sleep(.8)
#         for season in regular_seasons:
#             if season["MIN"] > 10.0:
#                 player_data["seasons"].append(season["SEASON_ID"])
#         if len(player_data["seasons"]) > 0:
#             position = commonplayerinfo.CommonPlayerInfo(player["PERSON_ID"]).get_normalized_dict()["CommonPlayerInfo"][0]["POSITION"]
#             position=position[0]
#             player_data["position"]=position
#             db.players.update_one({"playerID": season["PLAYER_ID"]}, {"$set":player_data}, True)
#             count+=1
#             print(str(count) + ": " + player["DISPLAY_FIRST_LAST"] + " " + position)
    

In [ ]:
def game_exists(games, game_id):
    for game in games:
        if game["gameID"] == game_id:
            return True
    return False

In [ ]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import boxscoretraditionalv2
from nba_api.stats.endpoints import boxscoresummaryv2
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.endpoints import boxscoremiscv2
import sys

sleeptime = .6

attempt = 0
count=960
for attempt in range(5):
    print("attempt: " + str(attempt))
    try:
        gamefinder = leaguegamefinder.LeagueGameFinder(league_id_nullable="00", season_nullable="2021-22", season_type_nullable="Regular Season")
        games = gamefinder.get_normalized_dict()["LeagueGameFinderResults"]
        print(len(games), " total games")
        games = [game for game in games if int(game["SEASON_ID"][1:]) >= 2021]
        games = games[count:]
        print(len(games), " recent games (2021)")
        playersCur = db.players.find()
        players_found = []
        for p in playersCur:
            players_found.append(p)
        print(len(players_found), " players found")
        
        for g in games:
            count+=1
            print(count)
            time.sleep(sleeptime)
            boxscorefinder = boxscoretraditionalv2.BoxScoreTraditionalV2(g["GAME_ID"]).get_normalized_dict()
            time.sleep(sleeptime)
            summary = boxscoresummaryv2.BoxScoreSummaryV2(g["GAME_ID"]).get_normalized_dict()["GameSummary"][0]
            time.sleep(sleeptime)
            advanced = boxscoreadvancedv2.BoxScoreAdvancedV2(g["GAME_ID"]).get_normalized_dict()
            time.sleep(sleeptime)
            misc = boxscoremiscv2.BoxScoreMiscV2(g["GAME_ID"]).get_normalized_dict()

            season = summary["SEASON"] + "-"+ str(int(summary["SEASON"][2:])+1)
            #players stats
            for game in boxscorefinder["PlayerStats"]:
                #if the player played in the game
                if game["MIN"] != None:
                    #check if player exists in the db
                    player_exists = False
                    for player in players_found:
                        if player["playerID"] == game["PLAYER_ID"]:
                            player_exists = True
                            break
                    if player_exists == False:
                        print(game["PLAYER_NAME"] + " not in DB")
                        continue
                    
                    team1 = lookupTeam(g["MATCHUP"].split(" ")[0])['id']
                    team2 = lookupTeam(g["MATCHUP"].split(" ")[2])['id']
                    home_or_away = ""
                    team = ""
                    opponent = ""
                    if team1 == game["TEAM_ID"]:
                        team = team1
                        opponent = team2
                        if g["MATCHUP"].split(" ")[1] == "@":
                            #team1 is away
                            home_or_away = "away"
                        else:
                            #team1 is home
                            home_or_away = "home"
                    elif team2 == game["TEAM_ID"]:
                        team = team2
                        opponent = team1
                        if g["MATCHUP"].split(" ")[1] == "@":
                            #team1 is away
                            home_or_away = "home"
                        else:
                            #team1 is home
                            home_or_away = "away"
                    else:
                        print("ERROR: Team not found")
                    game_data = {
                        "assists":game["AST"], 
                        "date": g["GAME_DATE"],
                        "defensive_rebounds":game["DREB"], 
                        "field_goal_percentage":game["FG_PCT"], 
                        "field_goals_attempted":game["FGA"], 
                        "field_goals_made":game["FGM"], 
                        "free_throws_attempted":game["FTA"],
                        "free_throws_made":game["FTM"], 
                        "free_throws_percentage":game["FT_PCT"],
                        "gameID":game["GAME_ID"],
                        "home_or_away":home_or_away,
                        "minutes": game["MIN"],
                        "offensive_rebounds":game["OREB"], 
                        "opponent":opponent,
                        "personal_fouls":game["PF"], 
                        "player":game["PLAYER_ID"],
                        "blocks":game["BLK"],
                        "steals":game["STL"],
                        "points": game["PTS"],
                        "season": season,
                        "team": team,
                        "three_point_percentage":game["FG3_PCT"], 
                        "three_pointers_attempted":game["FG3A"], 
                        "three_pointers_made":game["FG3M"], 
                        "total_rebounds":game["REB"], 
                        "turnovers":game["TO"], 
                    }
                    for advanced_game in advanced["PlayerStats"]:
                        if game_data['player'] == advanced_game["PLAYER_ID"]:
                            game_data["assist_percentage"] = advanced_game["AST_PCT"]
                            game_data["defensive_rebound_percentage"] = advanced_game["DREB_PCT"]
                            game_data["effective_field_goal_percentage"] = advanced_game["EFG_PCT"]
                            game_data["offensive_rebound_percentage"] = advanced_game["OREB_PCT"]
                            game_data["true_shooting_percentage"] = advanced_game["TS_PCT"]
                            game_data["usage"] = advanced_game["USG_PCT"]
                    for misc_game in misc["sqlPlayersMisc"]:
                        if game_data['player'] == misc_game["PLAYER_ID"]:
                            game_data["personal_fouls_drawn"] = misc_game["PFD"]
                    db.games.replace_one({"player":game_data["player"], "gameID": game_data["gameID"]}, game_data, True)
                    # print("\t", count, game["PLAYER_NAME"], "[", game_data["team"], "] vs [", game_data["opponent"], "]", game_data["season"], game_data["date"], game_data["minutes"], game_data["points"])
    except Exception as e:
        print("failed on attempt: " + str(attempt))
        print(e)
        print(game)
        count-=2
        time.sleep(sleeptime*10*attempt)